In [32]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [33]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [34]:
# Open Sheet
sheet = client.open_by_key("1Feh5lJStiEkMjT0qkLl8yu81Tbi1WjIeILcxrk13FsI") # Master Sheets ( Dimensions Sheet)

#### Extracting Masters from GoogleSheetBackend Masters

In [35]:
StoreMaster = sheet.worksheet("StoreMaster")
SOSTargetMaster = sheet.worksheet("SOSTargetMaster")
ProductMaster = sheet.worksheet("ProductMaster")
Overall_Master_Product = sheet.worksheet("Overall_Master_Product")

NPF_Master_SOS = sheet.worksheet("NPF_Master_SOS")


In [36]:
StoreMaster = get_as_dataframe(StoreMaster)
SOSTargetMaster = get_as_dataframe(SOSTargetMaster)
ProductMaster = get_as_dataframe(ProductMaster)

Overall_Master_Product = get_as_dataframe(Overall_Master_Product)
NPF_Master_SOS = get_as_dataframe(NPF_Master_SOS)


In [37]:
StoreMasterRevised = StoreMaster[
    ['LIS Store Code', 'Old DT Store Code', 'DT Store Code', 'Amazing ID', 'Emp User ID','LIS User Id','Employee Name']
]

StoreMasterRevised = StoreMasterRevised.rename(
    columns={
        'LIS Store Code': 'New_DT_Store_Code',
        'Old DT Store Code': 'OLD_DT_Store_Code',
        'DT Store Code': 'LIS_StoreCode',
        'Amazing ID': 'Old_Amazing_EMP_ID',
        'Emp User ID': 'New_Amazing_EMP_ID',
        'LIS User Id':'LISUserId',
        'Employee Name':'EmployeeName'
    }
)

In [38]:
## NEED TO HAVE CAMP ID ALWYAS IN UPPER CASE IN MASTER ANN COVERAGE AND SO ON

StoreMasterRevised['New_Amazing_EMP_ID'] = (
    StoreMasterRevised['New_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)

StoreMasterRevised['Old_Amazing_EMP_ID'] = (
    StoreMasterRevised['Old_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)

In [39]:
StoreMasterRevised.head()

New_DT_Store_Code OLD_DT_Store_Code  LIS_StoreCode Old_Amazing_EMP_ID  \
0       DT02DMA2001       DT02DMA2001  CLG-Store-497               X157   
1       DT02DMA2003       DT02DMA2003  CLG-Store-491          IN0028M57   
2       DT02DMA2005       DT02DMA2005  CLG-Store-494         IN0028M154   
3         DT02MET54       DT02MET54-1  CLG-Store-499         IN0028M475   
4       DT02RRLFRDF       DT02RRLFRDF  CLG-Store-496         IN0028M326   

  New_Amazing_EMP_ID    LISUserId    EmployeeName  
0            CAMP059  CLG-ISP-507    Mukesh Kumar  
1            CAMP061  CLG-ISP-926  Shankar Mandal  
2            CAMP062  CLG-ISP-504    Sachin Kumar  
3            CAMP112  CLG-ISP-509      Firoz Alam  
4            CAMP132  CLG-ISP-498           Vimal

In [40]:
StoreMasterRevised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   New_DT_Store_Code   837 non-null    object
 1   OLD_DT_Store_Code   808 non-null    object
 2   LIS_StoreCode       837 non-null    object
 3   Old_Amazing_EMP_ID  837 non-null    object
 4   New_Amazing_EMP_ID  837 non-null    object
 5   LISUserId           837 non-null    object
 6   EmployeeName        837 non-null    object
dtypes: object(7)
memory usage: 45.9+ KB


#### Plano

In [41]:
plano=pd.read_csv('Plano.csv', usecols = ['Category Type','Route Date','Status','Store Code','Store Name'])
# plano=pd.read_csv('Plano.csv')

In [42]:
# plano.columns
# ['Branch Classification', 'Branch', 'Region', 'Area', 'Zone', 'Channel',
#        'RE', 'Chain/Non Chain', 'Parent/Child', 'OCC/Non OCC', 'Sub RE',
#        'RE Type', 'Is Metro', 'Urban/Rural', 'Store Type', 'Store Code',
#        'Store Name', 'Route Date', 'Zing/AmaZing', 'Submission Type',
#        'Category Type', 'Rule Category', 'NPG Code Rule 1',
#        'NPG Description Rule 1', 'NPG Code Rule 2', 'NPG Description Rule 2',
#        'Status']

In [43]:
plano.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69968 entries, 0 to 69967
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Store Code     69968 non-null  object
 1   Store Name     69968 non-null  object
 2   Route Date     69968 non-null  object
 3   Category Type  69968 non-null  object
 4   Status         69968 non-null  object
dtypes: object(5)
memory usage: 2.7+ MB


In [44]:
plano.head()

Store Code                         Store Name Route Date Category Type  \
0    C001803249                   Suriya Hypermart  08-Jan-26    Toothpaste   
1    C002338970                  Shubham Ji Trader  08-Jan-26    Toothpaste   
2    C000465192     Kalwa Madhywarti G.s.s.m. No.2  08-Jan-26    Toothpaste   
3   DT01RRLFR03                     Infinity Malad  08-Jan-26    Toothpaste   
4  DT04APO15755  Apollo Pharmaapo-b.s.k. 3rd Stage  08-Jan-26    Toothpaste   

         Status  
0      Followed  
1      Followed  
2      Followed  
3      Followed  
4  Not Followed

In [45]:
plano['SliderStatus'] = np.where(plano['Status'] == 'Followed', 1, 0)

In [46]:
plano_master = pd.merge(plano,StoreMasterRevised,left_on = 'Store Code', right_on = 'New_DT_Store_Code', how = 'inner')

In [47]:
# plano_master.head()

In [48]:
plano_master['ReasonForNonAvailability'] = np.where(plano_master['Route Date'].isnull(), 0, 1)

In [49]:
plano_master.head()

Store Code        Store Name Route Date Category Type        Status  \
0  DT01RRLFR03    Infinity Malad  08-Jan-26    Toothpaste      Followed   
1  DT02FKW4703   C & C Zirakhpur  08-Jan-26    Toothbrush  Not Followed   
2   DT02SMK104      Sarita Vihar  08-Jan-26    Toothbrush  Not Followed   
3  DT02RRLFRGK  City Square Mall  08-Jan-26    Toothpaste      Followed   
4  DT01RRL6221        D-victoria  08-Jan-26    Toothpaste      Followed   

   SliderStatus New_DT_Store_Code OLD_DT_Store_Code  LIS_StoreCode  \
0             1       DT01RRLFR03       DT01RRLFR03  CLG-Store-341   
1             0       DT02FKW4703       DT02WMT4703  CLG-Store-420   
2             0        DT02SMK104       DT02SMK0104  CLG-Store-406   
3             1       DT02RRLFRGK       DT02RRLFRGK  CLG-Store-437   
4             1       DT01RRL6221       DT01RRL6221  CLG-Store-348   

  Old_Amazing_EMP_ID New_Amazing_EMP_ID     LISUserId           EmployeeName  \
0         IN0028M235            CAMP662   CLG-ISP-347  Karishma Sunil Sawant   
1               X310            CAMP102  CLG-ISP-1358                Shubham   
2         IN0028M193            CAMP182   CLG-ISP-415              Raj Kumar   
3               X876            CAMP146   CLG-ISP-447           Sushil Kumar   
4               X742            CAMP659   CLG-ISP-354              RAJA BABU   

   ReasonForNonAvailability  
0                         1  
1                         1  
2                         1  
3                         1  
4                         1

In [51]:
final_plano_master = plano_master[['Route Date','New_Amazing_EMP_ID','LISUserId','EmployeeName','New_DT_Store_Code','Store Name','SliderStatus','Category Type']]

In [52]:
final_plano_master.head()

Route Date New_Amazing_EMP_ID     LISUserId           EmployeeName  \
0  08-Jan-26            CAMP662   CLG-ISP-347  Karishma Sunil Sawant   
1  08-Jan-26            CAMP102  CLG-ISP-1358                Shubham   
2  08-Jan-26            CAMP182   CLG-ISP-415              Raj Kumar   
3  08-Jan-26            CAMP146   CLG-ISP-447           Sushil Kumar   
4  08-Jan-26            CAMP659   CLG-ISP-354              RAJA BABU   

  New_DT_Store_Code        Store Name  SliderStatus Category Type  
0       DT01RRLFR03    Infinity Malad             1    Toothpaste  
1       DT02FKW4703   C & C Zirakhpur             0    Toothbrush  
2        DT02SMK104      Sarita Vihar             0    Toothbrush  
3       DT02RRLFRGK  City Square Mall             1    Toothpaste  
4       DT01RRL6221        D-victoria             1    Toothpaste

In [55]:
# final_plano_masterx = final_plano_master[final_plano_master['New_Amazing_EMP_ID']=="CAMP012"]
# final_plano_masterx.head(10)

Route Date New_Amazing_EMP_ID   LISUserId       EmployeeName  \
233   08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   
408   08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   
410   08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   
582   08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   
800   08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   
811   08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   
1060  08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   
1147  08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   
1449  08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   
1718  08-Jan-26            CAMP012  CLG-ISP-31  Amit Kumar Mishra   

     New_DT_Store_Code      Store Name  SliderStatus Category Type  
233        DT03RRLTW60  Reliance_patia             1    Toothpaste  
408        DT03RRLTW60  Reliance_patia             1    Toothpaste  
410        DT03RRLTW60  Reliance_patia             1    Toothpaste  
582        DT03RRLTW60  Reliance_patia             1    Toothpaste  
800        DT03RRLTW60  Reliance_patia             1    Toothbrush  
811        DT03RRLTW60  Reliance_patia             1    Toothpaste  
1060       DT03RRLTW60  Reliance_patia             1    Toothpaste  
1147       DT03RRLTW60  Reliance_patia             1    Toothpaste  
1449       DT03RRLTW60  Reliance_patia             1    Toothpaste  
1718       DT03RRLTW60  Reliance_patia             0    Toothpaste

In [56]:
final_plano_master = final_plano_master.drop_duplicates()

In [57]:
final_plano_master.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4401 entries, 0 to 22600
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Route Date          4401 non-null   object
 1   New_Amazing_EMP_ID  4401 non-null   object
 2   LISUserId           4401 non-null   object
 3   EmployeeName        4401 non-null   object
 4   New_DT_Store_Code   4401 non-null   object
 5   Store Name          4401 non-null   object
 6   SliderStatus        4401 non-null   int32 
 7   Category Type       4401 non-null   object
dtypes: int32(1), object(7)
memory usage: 292.3+ KB
